# Answering Business Questions Using SQL

## Introduction and Schema Diagram

For this project, let's imagine we are a group of Business Analysts that have been employed to help a fictional music store named Chinook answer some questions they have about their business. 

The company has generously provided us with Chinook database and its relational schema for us to work with. The schema diagram is as follows:

![d](chinook_er_diagram.png)

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql 
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * FROM album LIMIT 3;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2


In [4]:
%%sql
SELECT * FROM invoice LIMIT 3;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98


## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and we have been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre |
| ----------- | ----- |
| Regal       | Hip-Hop |
| Red Tone    | Punk  |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [5]:
%%sql
WITH usa_sales AS (
    SELECT *
    FROM customer c
        JOIN invoice i ON c.customer_id = i.customer_id
        JOIN invoice_line il ON i.invoice_id = il.invoice_id
    WHERE c.country = 'USA'
)

SELECT 
    g.name genre,
    COUNT(t.track_id) num_tracks,
    CAST(COUNT(t.track_id) AS FLOAT) * 100
            / (SELECT COUNT(*) FROM usa_sales) perc_tracks
FROM usa_sales us
    JOIN track t ON us.track_id = t.track_id
    JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1 
ORDER BY 2 DESC;

Done.


genre,num_tracks,perc_tracks
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.4253092293054235
Alternative,35,3.330161750713606
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


Based on the results of the query we just executed above, we should purchase the albums from Red Tone, Slim Jim Bites, and Meteor and the Girls.

It is important to note, however, that the genres of these three artists only make up about 17% of the total sales which suggests we should be purchasing albums from artists within the 'Rock' genre.

## Analysing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [6]:
%%sql
WITH employee_sales AS (
    SELECT * 
    FROM employee e
        JOIN customer c ON e.employee_id = c.support_rep_id
        JOIN invoice i ON c.customer_id = i.customer_id
)

SELECT 
    first_name || ' ' || last_name employee_name,
    hire_date,
    SUM(total) tot_sales
FROM employee_sales
GROUP BY 1
ORDER BY 3 DESC;

Done.


employee_name,hire_date,tot_sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


We see that there are three sales support agents at Chinook and that total dollar amount of sales made by each of them appears to be closely related to how long they have worked there. That is, the early they joined the company as an employee, the more sales they tend to make which is what we would intuitively expect.

## Analysing Sales by Country

We would now like to analyse sales data for customers from each country individually. For the purposes of our analysis, we will use the country value from the `customers` table and ignore the country from the billing address in the `invoice` table.

For each country, we will compute the following summary metrics:

- Total number of customers
- Total value of sales
- Average value of sales per customer
- Average order value

Due to the fact there are numerous countries with only one customer who made purchases from Chinook, we will group these customers under a separate category named `Other` in our analysis.

In [7]:
%%sql
WITH country_with_other AS (
    SELECT 
        CASE WHEN 
            (SELECT COUNT(*)
             FROM customer
             WHERE country = c.country) = 1
        THEN 'Other' ELSE c.country
        END AS country,
        c.customer_id,
        il.*
    FROM customer c
        JOIN invoice i ON c.customer_id = i.customer_id
        JOIN invoice_line il ON i.invoice_id = il.invoice_id
)

SELECT 
    country,
    num_customers,
    total_amt,
    avg_cust_amt,
    avg_inv_amt
FROM (
    SELECT
        country,
        COUNT(DISTINCT customer_id) num_customers,
        SUM(unit_price) total_amt,
        SUM(unit_price) / 
             (SELECT COUNT(DISTINCT customer_id)) avg_cust_amt,
        SUM(unit_price) / 
             (SELECT COUNT(DISTINCT invoice_id)) avg_inv_amt,
        CASE 
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS sort_other
    FROM country_with_other
    GROUP BY country
    ORDER BY 
        sort_other, 
        num_customers DESC
    );


Done.


country,num_customers,total_amt,avg_cust_amt,avg_inv_amt
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486


## Albums vs. Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- Purchase a whole album
- Purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [8]:
%%sql
WITH invoice_first_track AS 
(
    SELECT
        il.invoice_id,
        MIN(il.track_id) first_track
    FROM invoice_line il
    GROUP BY il.invoice_id
)

SELECT 
    album_purchase,
    COUNT(invoice_id) num_invoices,
    CAST(COUNT(invoice_id) AS FLOAT) /
            (SELECT COUNT(invoice_id) FROM invoice) perc_invoices
FROM (
    SELECT 
        ifs.*,
        CASE
            WHEN
                (
                    SELECT il.track_id FROM invoice_line il
                    WHERE il.invoice_id = ifs.invoice_id

                    EXCEPT

                    SELECT t.track_id FROM track t
                    WHERE t.album_id = (
                        SELECT t2.album_id FROM track t2
                        WHERE t2.track_id = ifs.first_track
                    ) 
                ) 
                    IS NULL

                AND

                (
                    SELECT t.track_id FROM track t
                    WHERE t.album_id = (
                        SELECT t2.album_id FROM track t2
                        WHERE t2.track_id = ifs.first_track
                    )

                    EXCEPT

                    SELECT il.track_id FROM invoice_line il
                    WHERE il.invoice_id = ifs.invoice_id
                )
                    IS NULL
            THEN 'Yes' ELSE 'No'
        END AS album_purchase
    FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,num_invoices,perc_invoices
No,500,0.8143322475570033
Yes,114,0.18566775244299674


We see that only about 18% of purchases are for entire albums as opposed to individual tracks. Based on this, I would recommend that Chinook continues to purchase full albums from record labels because about one-fifth of the company's revenue will be lost if only individual tracks are purchased.